In [1]:
import os
import sys
while not os.getcwd().endswith('ml'):
    os.chdir('..')
sys.path.insert(0, os.getcwd())

In [2]:
import math
import copy
import cv2
import numpy as np
import random
from shutil import copyfile
from pathlib import Path
from matplotlib import pyplot as plt
from kaggle_problems.rosneft_proppant.workspace.helpers import ColoredCircleContour

circleContour = ColoredCircleContour()

In [3]:
DATA_DIR = "kaggle_problems/rosneft_proppant/data/colored_main_area"
ORIGINAL_IMG_DIR = "kaggle_problems/rosneft_proppant/data/train"
DEBUG_IMG_DIR = "kaggle_problems/rosneft_proppant/data/debug"
CIRCLE_DIR = "kaggle_problems/rosneft_proppant/data/circles"

In [4]:
def get_circle_unicolor(img, x, y, r):
    x, y = y, x
    sub_img = get_masked_img(img, x, y, r)
    if (sub_img is None):
        return None
    
    sub_img = sub_img.astype(float)
    
    msk = circleContour.get_msk(r)
    
    cnt_in_circle = np.sum(msk[:, :, 0])
    e = np.sum(sub_img, axis=(0, 1)) / cnt_in_circle
    
    sub_img -= e[np.newaxis, np.newaxis, :]
    sub_img *= msk 
    
    d = math.sqrt(np.sum(sub_img ** 2) / cnt_in_circle / 3)
    return d

def is_circle_unicolor(img, x, y, r):
    value = get_circle_unicolor(img, x, y, r)
    if value is None:
        return False
    return value < 20

def in_range(l, s, r):
    return l <= s and s < r

In [5]:
def get_masked_img(img, x, y, r):
    x_min = x - r
    x_max = x + r + 1
    y_min = y - r
    y_max = y + r + 1
    if (not in_range(0, x_min, img.shape[0])) or \
            (not in_range(0, x_max, img.shape[0])) or \
            (not in_range(0, y_min, img.shape[1])) or \
            (not in_range(0, y_max, img.shape[1])):
        return None

    msk = circleContour.get_msk(r)
    sub_img = img[x_min:x_max, y_min:y_max]

    sub_img = (sub_img * msk).astype(dtype=int)
    return sub_img

In [6]:
def is_colored_circle(img, x, y, r):
    x, y = y, x
    sub_img = get_masked_img(img, x, y, r)
    if (sub_img is None):
        return False
    
    sub_img = sub_img.astype(float)
    msk = helpers.circleContour.get_msk(r)
    
    tmp = np.empty(shape=sub_img.shape)
    tmp[:, :, 0] = np.abs(sub_img[:, :, 0] - sub_img[:, :, 1])
    tmp[:, :, 1] = np.abs(sub_img[:, :, 1] - sub_img[:, :, 2])
    tmp[:, :, 2] = np.abs(sub_img[:, :, 2] - sub_img[:, :, 0])
    
    sub_img = tmp
    
    cnt_in_circle = np.sum(msk)
    
    e = np.sum(sub_img) / cnt_in_circle
    
    sub_img -= e
    sub_img *= msk 
    
    d = np.sum(sub_img ** 2) / cnt_in_circle
    #sreturn d
    return d > 3000
    
    

In [7]:
def get_brith_circle(img, hsv_img, x, y, r):
    x, y = y, x
    hsv_sub_img = get_masked_img(hsv_img, x, y, r)
    if (hsv_sub_img is None):
        return False
    
    hsv_sub_img = hsv_sub_img.astype(float)
    msk = circleContour.get_msk(r)
    
    mean = np.sum(hsv_sub_img[:, :, 1]) / np.sum(msk)
    return mean
    
def is_brith_circle(img, hsv_img, x, y, r): # Яркость
    return get_brith_circle(img, hsv_img, x, y, r) > 40


In [8]:
def get_gray_around_circle(img, hsv_img, x, y, r):
    x, y = y, x
    hsv_sub_img = get_masked_img(hsv_img, x, y, r)
    hsv_sub_img2 = get_masked_img(hsv_img, x, y, r + 2)
    if (hsv_sub_img is None or hsv_sub_img2 is None):
        return False
    
    hsv_sub_img = hsv_sub_img.astype(float)
    msk = circleContour.get_msk(r)
    msk2 = circleContour.get_msk(r + 2)
    
    cnt = np.sum(msk)
    cnt2 = np.sum(msk2)
    
    _sum = np.sum(hsv_sub_img[:, :, 1])
    _sum2 = np.sum(hsv_sub_img2[:, :, 1])
    return (_sum2 - _sum) / (cnt2 - cnt)
    
def is_gray_around_circle(img, hsv_img, x, y, r): # Яркость
    return get_gray_around_circle(img, hsv_img, x, y, r) < 15


In [9]:
def process_color_img(original_img, img_number):
    result_circles = []
    debug_dir = "{}/{}".format(DEBUG_IMG_DIR, img_number)
    
    Path(debug_dir).mkdir(exist_ok=True, parents=True) # create debug dir
    
    img_with_circle = copy.deepcopy(original_img)
    img = copy.deepcopy(original_img)
    
    gaussian_3 = cv2.GaussianBlur(img, (3, 3), 20.0)
    img = cv2.addWeighted(img, 1.5, gaussian_3, -0.5, 0, img)
    
    hsv_img = cv2.cvtColor(original_img, cv2.COLOR_RGB2HSV)
    
 #   cv2.imwrite("{}/hsv_img.jpg".format(debug_dir), hsv_img)
#     i1 = copy.deepcopy(hsv_img)
#     i1[hsv_img[:, :, 0] < 150, 1:3] = 0 
    
#     i1 = cv2.cvtColor(i1, cv2.COLOR_HSV2RGB)
#     cv2.imwrite("{}/i1.jpg".format(debug_dir), i1)
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #gray = hsv_img[:, :, 0]
    
#     plt.hist2d(hsv_img[:, :, 0].reshape(hsv_img.shape[0] * hsv_img.shape[1]), 
#                hsv_img[:, :, 2].reshape(hsv_img.shape[0] * hsv_img.shape[1]),
#                bins=20)
#     plt.show()
    
#     cv2.imwrite("{}/original_img.jpg".format(debug_dir), original_img)
#     cv2.imwrite("{}/img.jpg".format(debug_dir), img)
#     cv2.imwrite("{}/gray.jpg".format(debug_dir), gray)

    
    it = 0
    for r in range(40, 10, -1):
        print('-' * 30)
        # Выделение кругов
        circles = cv2.HoughCircles(gray,cv2.HOUGH_GRADIENT,0.5,minDist=1.7 * r,
                                param1=200,param2=1,minRadius=r,maxRadius=r)
        circles = circles[0]
        print("Circle count: {}. r: {}".format(len(circles), r))
        filtered_circle = [(int(circle[0]), int(circle[1]), int(circle[2])) for circle in circles]
        
        filtered_circle = [circle for circle in filtered_circle if is_brith_circle(img, hsv_img, circle[0], circle[1], circle[2])]
        print("After first filter: {}. r: {}".format(len(filtered_circle), r))
        
#         graynes = [get_gray_around_circle(img, hsv_img, circle[0], circle[1], circle[2]) for circle in filtered_circle]
    
#         plt.hist(graynes, bins=10)
#         plt.show()

        filtered_circle = [circle for circle in filtered_circle if is_gray_around_circle(img, hsv_img, circle[0], circle[1], circle[2])]
        
        print("After second filter: {}. r: {}".format(len(filtered_circle), r))
        
        #tmp = [get_circle_unicolor(img, circle[0], circle[1], circle[2]) for circle in filtered_circle]

        #filtered_circle = [circle for circle in filtered_circle if is_circle_unicolor(img, circle[0], circle[1], circle[2])]
        #print("After second filter: {}. r: {}".format(len(filtered_circle), r))
        
        for circle in filtered_circle:
            cv2.circle(img,(circle[0],circle[1]),circle[2],(255, 255, 255),-1)
            cv2.circle(gray,(circle[0],circle[1]),circle[2],(255, 255, 255),-1)
            cv2.circle(img_with_circle,(circle[0],circle[1]),circle[2], (0, 255, 0),1)

#         cv2.imwrite("{}/img_{}.jpg".format(debug_dir, it), img)
#         cv2.imwrite("{}/img_with_circle_{}.jpg".format(debug_dir, it), img_with_circle)
#         cv2.imwrite("{}/gray_{}.jpg".format(debug_dir, it), gray)
#         cv2.imwrite("{}/hsv_img_{}.jpg".format(debug_dir, it), hsv_img[:, :, 1])
        
        it += 1
        result_circles.extend(filtered_circle)
        
    return result_circles



In [10]:
all_img = [img for img in os.listdir(DATA_DIR) if img.endswith('.jpg')]

for img_name in all_img:
    print(img_name)
    img_number, _ = img_name.split('.')
    original_img = cv2.imread("{}/{}".format(DATA_DIR, img_name))

    circles = process_color_img(original_img, img_number)
    for circle, i in zip(circles, range(len(circles))):
        circle_img = get_masked_img(original_img, circle[1], circle[0], circle[2])
        if circle_img is not None:
            cv2.imwrite("{}/{}_{}.jpg".format(CIRCLE_DIR, img_number, i), circle_img)

189.jpg
------------------------------
Circle count: 831. r: 40
After first filter: 242. r: 40
After second filter: 0. r: 40
------------------------------
Circle count: 867. r: 39
After first filter: 261. r: 39
After second filter: 0. r: 39
------------------------------
Circle count: 923. r: 38
After first filter: 287. r: 38
After second filter: 0. r: 38
------------------------------
Circle count: 968. r: 37
After first filter: 293. r: 37
After second filter: 0. r: 37
------------------------------
Circle count: 1004. r: 36
After first filter: 308. r: 36
After second filter: 0. r: 36
------------------------------
Circle count: 1050. r: 35
After first filter: 334. r: 35
After second filter: 0. r: 35
------------------------------
Circle count: 1093. r: 34
After first filter: 351. r: 34
After second filter: 0. r: 34
------------------------------
Circle count: 1149. r: 33
After first filter: 382. r: 33
After second filter: 0. r: 33
------------------------------
Circle count: 1215. r

Circle count: 1624. r: 31
After first filter: 896. r: 31
After second filter: 0. r: 31
------------------------------
Circle count: 1689. r: 30
After first filter: 941. r: 30
After second filter: 0. r: 30
------------------------------
Circle count: 1818. r: 29
After first filter: 1025. r: 29
After second filter: 0. r: 29
------------------------------
Circle count: 1926. r: 28
After first filter: 1074. r: 28
After second filter: 0. r: 28
------------------------------
Circle count: 2049. r: 27
After first filter: 1165. r: 27
After second filter: 0. r: 27
------------------------------
Circle count: 2183. r: 26
After first filter: 1248. r: 26
After second filter: 0. r: 26
------------------------------
Circle count: 2335. r: 25
After first filter: 1347. r: 25
After second filter: 0. r: 25
------------------------------
Circle count: 2499. r: 24
After first filter: 1469. r: 24
After second filter: 0. r: 24
------------------------------
Circle count: 2701. r: 23
After first filter: 1587

Circle count: 1941. r: 21
After first filter: 1146. r: 21
After second filter: 0. r: 21
------------------------------
Circle count: 2130. r: 20
After first filter: 1267. r: 20
After second filter: 0. r: 20
------------------------------
Circle count: 2277. r: 19
After first filter: 1358. r: 19
After second filter: 0. r: 19
------------------------------
Circle count: 2521. r: 18
After first filter: 1544. r: 18
After second filter: 2. r: 18
------------------------------
Circle count: 2732. r: 17
After first filter: 1699. r: 17
After second filter: 2. r: 17
------------------------------
Circle count: 2988. r: 16
After first filter: 1888. r: 16
After second filter: 2. r: 16
------------------------------
Circle count: 3339. r: 15
After first filter: 2143. r: 15
After second filter: 4. r: 15
------------------------------
Circle count: 3778. r: 14
After first filter: 2476. r: 14
After second filter: 4. r: 14
------------------------------
Circle count: 4251. r: 13
After first filter: 28

After second filter: 10. r: 12
------------------------------
Circle count: 5262. r: 11
After first filter: 2514. r: 11
After second filter: 13. r: 11
598.jpg
------------------------------
Circle count: 1489. r: 40
After first filter: 1079. r: 40
After second filter: 0. r: 40
------------------------------
Circle count: 1553. r: 39
After first filter: 1117. r: 39
After second filter: 0. r: 39
------------------------------
Circle count: 1633. r: 38
After first filter: 1193. r: 38
After second filter: 0. r: 38
------------------------------
Circle count: 1723. r: 37
After first filter: 1284. r: 37
After second filter: 0. r: 37
------------------------------
Circle count: 1798. r: 36
After first filter: 1329. r: 36
After second filter: 0. r: 36
------------------------------
Circle count: 1889. r: 35
After first filter: 1418. r: 35
After second filter: 0. r: 35
------------------------------
Circle count: 2007. r: 34
After first filter: 1500. r: 34
After second filter: 0. r: 34
--------

After second filter: 0. r: 33
------------------------------
Circle count: 1505. r: 32
After first filter: 1022. r: 32
After second filter: 0. r: 32
------------------------------
Circle count: 1595. r: 31
After first filter: 1095. r: 31
After second filter: 0. r: 31
------------------------------
Circle count: 1695. r: 30
After first filter: 1179. r: 30
After second filter: 0. r: 30
------------------------------
Circle count: 1789. r: 29
After first filter: 1260. r: 29
After second filter: 0. r: 29
------------------------------
Circle count: 1896. r: 28
After first filter: 1335. r: 28
After second filter: 0. r: 28
------------------------------
Circle count: 2007. r: 27
After first filter: 1442. r: 27
After second filter: 0. r: 27
------------------------------
Circle count: 2136. r: 26
After first filter: 1539. r: 26
After second filter: 0. r: 26
------------------------------
Circle count: 2276. r: 25
After first filter: 1668. r: 25
After second filter: 0. r: 25
------------------

...

Circle count: 2133. r: 26
After first filter: 993. r: 26
After second filter: 1. r: 26
------------------------------
Circle count: 2286. r: 25
After first filter: 1080. r: 25
After second filter: 4. r: 25
------------------------------
Circle count: 2462. r: 24
After first filter: 1192. r: 24
After second filter: 3. r: 24
------------------------------
Circle count: 2601. r: 23
After first filter: 1263. r: 23
After second filter: 4. r: 23
------------------------------
Circle count: 2815. r: 22
After first filter: 1406. r: 22
After second filter: 4. r: 22
------------------------------
Circle count: 3033. r: 21
After first filter: 1497. r: 21
After second filter: 3. r: 21
------------------------------
Circle count: 3308. r: 20
After first filter: 1689. r: 20
After second filter: 3. r: 20
------------------------------
Circle count: 3584. r: 19
After first filter: 1852. r: 19
After second filter: 1. r: 19
------------------------------
Circle count: 3911. r: 18
After first filter: 203

Circle count: 6231. r: 17
After first filter: 5415. r: 17
After second filter: 0. r: 17
------------------------------
Circle count: 6948. r: 16
After first filter: 6067. r: 16
After second filter: 1. r: 16
------------------------------
Circle count: 7790. r: 15
After first filter: 6822. r: 15
After second filter: 2. r: 15
------------------------------
Circle count: 9027. r: 14
After first filter: 7970. r: 14
After second filter: 4. r: 14
------------------------------
Circle count: 10494. r: 13
After first filter: 9321. r: 13
After second filter: 2. r: 13
------------------------------
Circle count: 12170. r: 12
After first filter: 10786. r: 12
After second filter: 0. r: 12
------------------------------
Circle count: 13590. r: 11
After first filter: 11945. r: 11
After second filter: 0. r: 11
636.jpg
------------------------------
Circle count: 697. r: 40
After first filter: 153. r: 40
After second filter: 0. r: 40
------------------------------
Circle count: 742. r: 39
After first 

Circle count: 797. r: 37
After first filter: 250. r: 37
After second filter: 0. r: 37
------------------------------
Circle count: 832. r: 36
After first filter: 266. r: 36
After second filter: 0. r: 36
------------------------------
Circle count: 874. r: 35
After first filter: 284. r: 35
After second filter: 0. r: 35
------------------------------
Circle count: 915. r: 34
After first filter: 306. r: 34
After second filter: 0. r: 34
------------------------------
Circle count: 952. r: 33
After first filter: 312. r: 33
After second filter: 0. r: 33
------------------------------
Circle count: 999. r: 32
After first filter: 336. r: 32
After second filter: 0. r: 32
------------------------------
Circle count: 1044. r: 31
After first filter: 347. r: 31
After second filter: 0. r: 31
------------------------------
Circle count: 1091. r: 30
After first filter: 360. r: 30
After second filter: 0. r: 30
------------------------------
Circle count: 1139. r: 29
After first filter: 412. r: 29
After

Circle count: 1225. r: 27
After first filter: 538. r: 27
After second filter: 0. r: 27
------------------------------
Circle count: 1301. r: 26
After first filter: 574. r: 26
After second filter: 0. r: 26
------------------------------
Circle count: 1379. r: 25
After first filter: 623. r: 25
After second filter: 0. r: 25
------------------------------
Circle count: 1485. r: 24
After first filter: 683. r: 24
After second filter: 0. r: 24
------------------------------
Circle count: 1574. r: 23
After first filter: 743. r: 23
After second filter: 0. r: 23
------------------------------
Circle count: 1669. r: 22
After first filter: 804. r: 22
After second filter: 1. r: 22
------------------------------
Circle count: 1806. r: 21
After first filter: 887. r: 21
After second filter: 0. r: 21
------------------------------
Circle count: 1973. r: 20
After first filter: 984. r: 20
After second filter: 1. r: 20
------------------------------
Circle count: 2120. r: 19
After first filter: 1086. r: 1

After second filter: 0. r: 18
------------------------------
Circle count: 4411. r: 17
After first filter: 2394. r: 17
After second filter: 0. r: 17
------------------------------
Circle count: 4870. r: 16
After first filter: 2687. r: 16
After second filter: 1. r: 16
------------------------------
Circle count: 5476. r: 15
After first filter: 3054. r: 15
After second filter: 0. r: 15
------------------------------
Circle count: 6153. r: 14
After first filter: 3489. r: 14
After second filter: 1. r: 14
------------------------------
Circle count: 6920. r: 13
After first filter: 3999. r: 13
After second filter: 2. r: 13
------------------------------
Circle count: 7861. r: 12
After first filter: 4661. r: 12
After second filter: 2. r: 12
------------------------------
Circle count: 9175. r: 11
After first filter: 5463. r: 11
After second filter: 0. r: 11
740.jpg
------------------------------
Circle count: 859. r: 40
After first filter: 306. r: 40
After second filter: 0. r: 40
------------

In [11]:
original_img = cv2.imread("{}/tmp/{}".format(DATA_DIR, img_name + ".png")) 
original_img = cv2.cvtColor(original_img, cv2.COLOR_RGB2HSV)
original_img[:, :, 0] = 255
original_img = cv2.cvtColor(original_img, cv2.COLOR_HSV2RGB)

cv2.imwrite("img.png", original_img)

error: OpenCV(4.4.0) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-rjwswi6i/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
v = np.reshape(original_img[:, :, 1], (original_img.shape[0] * original_img.shape[1]))
plt.hist(v, bins=range(0, 256))
plt.show()

In [ ]:
#H: 0 - 180, S: 0 - 255, V: 0 - 255